## Figure 3: Glacier inventory comparison between RGI6 and RGI7

In [ ]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely.geometry

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

cl = px.colors.qualitative.D3
os.chdir('/home/rooda/Dropbox/Patagonia')

## Data

In [ ]:
# data
RGI6   = gpd.read_file("GIS South/Glaciers/RGI6_v2.shp")
RGI7   = gpd.read_file("GIS South/Glaciers/RGI7_v2.shp")

RGI6["year"] = RGI6.BgnDate.str[0:4].astype("int16")
RGI7["year"] = RGI7.src_date.str[0:4].astype("int16")

# Zone: numeric to name (for the plot)
RGI6 = RGI6.replace({"Zone": {1:'PPY', 2:'PCA', 3:'NPI-E', 4:'NPI-W', 5:'SPI-N', 6:'SPI-C', 7:'SPI-S', 8:'GCN', 9:'CDI'}})
RGI7 = RGI7.replace({"Zone": {1:'PPY', 2:'PCA', 3:'NPI-E', 4:'NPI-W', 5:'SPI-N', 6:'SPI-C', 7:'SPI-S', 8:'GCN', 9:'CDI'}})

# polygons to points (centroid)
basins = gpd.read_file("GIS South/Basins_Patagonia_ice.shp")
basins = basins.set_index("ID")
basins["diff"] = ((basins['RGI7_area']/basins['RGI6_area'])-1)*100

names = ["Yelcho", "Baker", "Santa Cruz                             ", "Palena", "Grey", "Puelo", "Cisnes", "Aysen", "Pascua"]
basins.loc[basins.basin_area > 5000, "Name"] = names

RGI6_agg  = RGI6[["Zone","area_km2"]].groupby("Zone").sum()
RGI7_agg  = RGI7[["Zone","area_km2"]].groupby("Zone").sum()
diff_perc = (RGI7_agg.area_km2 / RGI6_agg.area_km2) - 1

diff_abs  = (RGI7_agg.area_km2 - RGI6_agg.area_km2).astype("int").tolist()
diff_abs  = [str(x)+' km<sup>2</sup>' for x in diff_abs]

In [ ]:
# basemap for background
geo_map = gpd.read_file("/home/rooda/Dropbox/ArcGIS/Chile/south_america.shp")
geo_map = geo_map[(geo_map.CC == "CI") | (geo_map.CC == "AR")]
geo_map = geo_map.dissolve(by='REGION')
geo_map["geometry"] = geo_map.simplify(0.01)

poly_gdf = shapely.geometry.Polygon([(-76, -55.7), (-76, -40.52), (-68.05, -40.52), (-68.05, -55.7), (-76, -55.8)])
poly_gdf = gpd.GeoDataFrame([1], geometry=[poly_gdf], crs=geo_map.crs)
geo_map = geo_map.clip(poly_gdf)

In [ ]:
# hydrological zone divides
geo_lines = gpd.read_file("GIS South/Basins_Patagonia_ice_divides.shp")

lats = []
lons = []

for feature in geo_lines.geometry:
    if isinstance(feature, shapely.geometry.linestring.LineString):
        linestrings = [feature]
    elif isinstance(feature, shapely.geometry.multilinestring.MultiLineString):
        linestrings = feature.geoms
    else:
        continue
    for linestring in linestrings:
        x, y = linestring.xy
        lats = np.append(lats, y)
        lons = np.append(lons, x)
        lats = np.append(lats, None)
        lons = np.append(lons, None)
        
lat_coords  = [-43.2, -45.95,  -46.9,  -47.55,   -49.2,   -50.5,   -52.0, -53.1, -54.8]
lon_coords  = [-71.2, -71.7,   -73.8,   -71.7,   -72.2,   -72.3,   -72.1, -72.8, -69.2]
names_zones = ["PPY", "PCA", "NPI-W", "NPI-E", "SPI-N", "SPI-C", "SPI-S", "GCN", "CDI"]
names  = ['<b>'+x+'</b>' for x in names_zones]

## Plot

In [ ]:
fig = make_subplots(rows=2, cols=3, horizontal_spacing = 0.06, vertical_spacing = 0.1,
                    specs=[[{"type": "scattergeo", "rowspan": 2}, {"type": "xy", "colspan": 2}, None],
                           [None,                                 {"type": "xy", "r": 0}, {"type": "xy", "l": 0}]])

# a) Map ----------------------------------------------------------------------------------------------------------

## Basemap
fig.add_trace(go.Choropleth(geojson = eval(geo_map['geometry'].to_json()),  locations = geo_map.index, z = geo_map['iso_num'], 
                            colorscale = ["#EAEAF2", "#EAEAF2"], showscale= False, marker_line_color ='white', marker_line_width=0.1), row=1, col=1)

## Added shapefiles
colorbar_config = dict(len=0.5, x=0.22, y= 0.75, title='Δ Area', ticksuffix = "%", thickness=20, tickfont=dict(size=13))
fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()),  locations = basins.index, z = basins["diff"], 
                            colorscale = [cl[1], "#ffe9ba" ,cl[0]], marker_line_color ='rgba(255,255,255, 0.6)', marker_line_width=0.5,
                            zmin = -100, zmax = 100, colorbar = colorbar_config), row=1, col=1)

## Added shapefiles (lines)
fig.add_trace(go.Scattergeo(lon = lons, lat = lats, mode = 'lines', line = dict(width = 0.7,color = 'black'), opacity = 0.5, showlegend = False),row=1, col=1)
fig.add_trace(go.Scattergeo(lon=lon_coords, lat=lat_coords, mode='text', text=names, textfont=dict(size=12, color = "rgba(0,0,0,0.6)"), showlegend = False),row=1, col=1)
fig.add_scattergeo(geojson = eval(basins['geometry'].to_json()), locations = basins.index, text = basins['Name'], mode = 'text', showlegend = False, 
                   textfont=dict(size=11, color = "rgba(0,0,0,0.3)"),row=1, col=1)

## Background
fig.update_geos(showframe = True, framecolor = "rgba(0,0,0,0.5)", framewidth = 1, lonaxis_range=[-76, -68], lataxis_range=[-55.8, -40.5], 
                bgcolor = "rgb(255,255,255)", showland = False, showcoastlines = False, showlakes = False)
fig.add_annotation(text="a)", font=dict(size=16), x=0.01, y=0.995,  xref = "paper", yref = "paper", showarrow=False)


# b) Difference by Zone --------------------------------------------------------------------------------------------------
colorbar_config_b = dict(title='Area by zone', outlinewidth = 0, ticksuffix = " km<sup>2</sup>", len=0.4, x=0.5, y= 0.79, thickness=20)
fig.add_trace(go.Bar(x=diff_perc.index, y=diff_perc.values, text = diff_abs,
                     marker= dict(color = diff_perc.values, colorscale = ["#ffb05d", "#ffe9ba", "#8eafb6"], showscale=False, colorbar = colorbar_config_b)), row=1, col=2)
fig.update_traces(showlegend=False, opacity=1, textfont_size=14, textangle=0, textposition="outside", cliponaxis=False, row=1, col=2)
fig.update_yaxes(title = "Difference per zone rel. to RGI6 (%)", tickangle = -90, dtick = 0.20, title_standoff = 0, zeroline = True, range = [-0.32,0.32], tickformat = ',.0%', row=1, col=2)
fig.update_xaxes(categoryorder='array', categoryarray= names_zones, row=1, col=2)
fig.update_layout(bargap=0.15)
fig.update_traces(textposition='auto', row=1, col=2)
fig.add_annotation(text="b)", font=dict(size=16), x=-0.35, y=0.29, showarrow=False, row=1, col=2)
fig.add_annotation(text="Absolute difference", font=dict(size=14), x=1, y=0.09, ax=40, showarrow=True, row=1, col=2)


# b) Area in log km2 --------------------------------------------------------------------------------------------------
fig.add_trace(go.Histogram(x=np.log10(RGI6.area_km2), histnorm='percent', marker_color= cl[1], name = "RGI6"), row=2, col=2)
fig.add_trace(go.Histogram(x=np.log10(RGI7.area_km2), histnorm='percent', marker_color= cl[0], name = "RGI7"), row=2, col=2)
fig.update_yaxes(title = "Percent of glacier number (%)", tickangle = -90, range = (0,4), dtick = 1, row=2, col=2)
fig.update_xaxes(title = "Area (log km<sup>2</sup>)", title_standoff = 0, tickmode = 'array', row=2, col=2)
fig.update_xaxes(tickvals = [-2, -1, 0, 1, 2], ticktext = ["10<sup>-2<sup>", "10<sup>-1<sup>", "10<sup>0<sup>","10<sup>1<sup>", "10<sup>2<sup>"], row=2, col=2)
fig.update_traces(opacity=0.7, row = 2, col = 2)
fig.add_annotation(text="c)", font=dict(size=16), x=-2, y=3.8, showarrow=False, row=2, col=2)
fig.add_annotation(text="Total number of glaciers:", font=dict(size=14), x=1.8, y=3.8, showarrow=False, row=2, col=2)
fig.add_annotation(text="• " + str(len(RGI6)) +" in RGI6", font=dict(size=14), x=2.1, y=3.5, showarrow=False, row=2, col=2)
fig.add_annotation(text="• " + str(len(RGI7)) +" in RGI7", font=dict(size=14), x=2.1, y=3.2, showarrow=False, row=2, col=2)


# c) RGI year ---------------------------------------------------------------------------------------------------------
RGI6a = RGI6[["year", "area_km2"]].groupby(["year"]).sum()*100 /  RGI6.area_km2.sum()
RGI7a = RGI7[["year", "area_km2"]].groupby(["year"]).sum()*100 /  RGI7.area_km2.sum()

fig.add_trace(go.Bar(x=RGI6a.index, y=RGI6a.area_km2, marker_color= cl[1], showlegend = False), row=2, col=3)
fig.add_trace(go.Bar(x=RGI7a.index, y=RGI7a.area_km2, marker_color= cl[0], showlegend = False), row=2, col=3)
fig.update_xaxes(title = "Acquisition date (year)", title_standoff = 0, dtick = 3, row=2, col=3)
fig.update_yaxes(title = "Percent of total glacier area (%)", range = (0,100), tickangle = -90, row=2, col=3)
fig.update_traces(opacity= 0.7, row = 2, col = 3)
fig.add_annotation(text="d)", font=dict(size=16), x=2000, y=95, showarrow=False, row=2, col=3)


# All ------------------------------------------------------------------------------------------------------------------
fig.update_xaxes(showline = True, linecolor = 'rgba(0,0,0,0.5)', linewidth = 1, ticks="outside", griddash = "dot", mirror=True)
fig.update_yaxes(showline = True, linecolor = 'rgba(0,0,0,0.5)', linewidth = 1, ticks="outside", griddash = "dot", mirror=True, title_standoff = 0)

fig.update_layout(font=dict(size=14), barmode='overlay', margin = dict(l=5, r=5, b=5, t=30), autosize=False, width = 1150, height = 700)
fig.update_layout(legend=dict(y=0.4, x=0.925, bgcolor = 'rgba(255,255,255,0.5)'))
fig.show()

fig.write_image("/home/rooda/Dropbox/Patagonia/MS2 Results/Figure_3_RGI.png", scale=4)

In [ ]:
(RGI7.year < 2001).sum()/len(RGI7)

In [ ]:
(RGI7.year < 2004).sum()/len(RGI7)

In [ ]:
RGI7[RGI7.year < 2004].area_km2.sum()/RGI7.area_km2.sum()

In [ ]:
basins[basins["diff"].abs() < 10].basin_area.sum()/basins.basin_area.sum()

In [ ]:
basins[basins["diff"].abs() < 10].RGI6_area.sum()/basins.RGI6_area.sum()

## Hexbins alternative for a)

In [ ]:
fig = ff.create_hexbin_mapbox(
    data_frame=RGI, lat="CenLat", lon="CenLon",
    nx_hexagon=15, opacity=0.7, labels={"color": "ΔGlaciers (n)"},
    min_count=1, color="number", agg_func=np.sum, color_continuous_scale=[(0, c2), (0.16, "white"), (1, c1)], 
    range_color = [-50,250], mapbox_style = "carto-positron")

fig.update_mapboxes(style='carto-positron', center={'lat': -48.5, 'lon': -72},  zoom=5.3)

fig.update_layout(font=dict(size=20), autosize=False, width=1000, height=1500)
fig.update_layout(margin = {'l':0.1,'r':0.1,'t':0.1,'b':0.1})
fig.show()

In [ ]:
fig = make_subplots(rows=2, cols=3, horizontal_spacing = 0.05, vertical_spacing = 0.1,
                    specs=[[{"type": "scattergeo", "rowspan": 2}, {"type": "xy", "colspan": 2}, None],
                           [None,                                 {"type": "xy", "r": 0}, {"type": "xy", "l": 0}]])

# b) Area in log km2 --------------------------------------------------------------------------------------------------
fig.add_trace(go.Histogram(x=np.log10(RGI6.area_km2), histnorm='percent', marker_color= cl[1], name = "RGI6"), row=2, col=2)
fig.add_trace(go.Histogram(x=np.log10(RGI7.area_km2), histnorm='percent', marker_color= cl[0], name = "RGI7"), row=2, col=2)
fig.update_yaxes(title = "Percent (%)", tickangle = -90, range = (0,4), dtick = 1, row=2, col=2)
fig.update_xaxes(title = "Area (log km<sup>2</sup>)", title_standoff = 0, tickmode = 'array', row=2, col=2)
fig.update_xaxes(tickvals = [-2, -1, 0, 1, 2], ticktext = ["10<sup>-2<sup>", "10<sup>-1<sup>", "10<sup>0<sup>","10<sup>1<sup>", "10<sup>2<sup>"], row=2, col=2)
fig.update_traces(opacity=0.7, row = 2, col = 2)
fig.add_annotation(text="c)", font=dict(size=16), x=-2, y=3.8, showarrow=False, row=2, col=2)
fig.add_annotation(text="Total number of glaciers:", font=dict(size=14), x=1.8, y=3.8, showarrow=False, row=2, col=2)
fig.add_annotation(text="• " + str(len(RGI6)) +" in RGI6", font=dict(size=14), x=2.1, y=3.5, showarrow=False, row=2, col=2)
fig.add_annotation(text="• " + str(len(RGI7)) +" in RGI7", font=dict(size=14), x=2.1, y=3.2, showarrow=False, row=2, col=2)

# All ------------------------------------------------------------------------------------------------------------------
fig.update_xaxes(showline = True, linecolor = 'rgba(0,0,0,0.5)', linewidth = 1, ticks="outside", griddash = "dot", mirror=True)
fig.update_yaxes(showline = True, linecolor = 'rgba(0,0,0,0.5)', linewidth = 1, ticks="outside", griddash = "dot", mirror=True, title_standoff = 0)

fig.update_layout(font=dict(size=14), barmode='overlay', plot_bgcolor="rgba(213,213,213,0.6)", margin = dict(l=5, r=5, b=5, t=30), autosize=False, width = 1150, height = 700)
fig.update_layout(legend=dict(y=0.4, x=0.925, bgcolor = 'rgba(255,255,255,0.5)'))
fig.show()